In [40]:
import pandas as pd
import matplotlib.pyplot as plt
from alpha_vantage.timeseries import TimeSeries
import time
import json
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
api_key = '7DOY154L31ZDC2PG'

In [30]:
def stocks(ticker):
    ticker = ticker.upper()
    ts = TimeSeries(key='7DOY154L31ZDC2PG', output_format='pandas')
    data, meta_data = ts.get_daily_adjusted(symbol=ticker, outputsize='full')
    df = pd.DataFrame(data)
   # df = df.reset_index(level=0)
    df = df.rename(columns={"4. close": "Close",
                        "7. dividend amount": "Dividend",
                        "2. high": "High",
                        "3. low": "Low",
                        "1. open": "Open",
                        "8. split coefficient": "Split Coefficient",
                        "6. volume": "Volume"})
    df = df.drop('5. adjusted close',1)
    
        # we will go from today to the past
    new = df.sort_index(ascending=False)

    split_coef = new['Split Coefficient'].shift(1
        ).fillna(1).cumprod()

    for col in ['Open', 'High', 'Low', 'Close']:
    	  new['Adj.' + col] = new[col] / split_coef
    new['Adj.Volume'] = split_coef * new['Volume']
    dividends=False
    if dividends:
        new['Adj. Dividends'] = new['Dividend Amount'] / split_coef
    new['Daily Change'] = new['Adj.Close'] - new['Adj.Open']
    return new.sort_index(ascending=True)


In [31]:
apple = stocks('aapl')

In [32]:
apple

Open     High       Low   Close       Volume  Dividend  \
date                                                                    
1999-11-01   80.000   80.690   77.3700   77.62    2487300.0       0.0   
1999-11-02   78.000   81.690   77.3100   80.25    3564600.0       0.0   
1999-11-03   81.620   83.250   81.0000   81.50    2932700.0       0.0   
1999-11-04   82.060   85.370   80.6200   83.62    3384700.0       0.0   
1999-11-05   84.620   88.370   84.0000   88.31    3721500.0       0.0   
...             ...      ...       ...     ...          ...       ...   
2021-11-16  149.940  151.488  149.3400  151.00   59256210.0       0.0   
2021-11-17  150.995  155.000  150.9900  153.49   88807000.0       0.0   
2021-11-18  153.710  158.670  153.0500  157.87  137827673.0       0.0   
2021-11-19  157.650  161.020  156.5328  160.55  117305597.0       0.0   
2021-11-22  161.680  165.700  161.0000  161.02  116197266.0       0.0   

            Split Coefficient    Adj.Open    Adj.High     Adj.Low   Adj.Close  \
date                                                                            
1999-11-01                1.0    0.714285    0.720446    0.690803    0.693035   
1999-11-02                1.0    0.696428    0.729374    0.690267    0.716517   
1999-11-03                1.0    0.728749    0.743303    0.723214    0.727678   
1999-11-04                1.0    0.732678    0.762231    0.719821    0.746606   
1999-11-05                1.0    0.755535    0.789017    0.749999    0.788481   
...                       ...         ...         ...         ...         ...   
2021-11-16                1.0  149.940000  151.488000  149.340000  151.000000   
2021-11-17                1.0  150.995000  155.000000  150.990000  153.490000   
2021-11-18                1.0  153.710000  158.670000  153.050000  157.870000   
2021-11-19                1.0  157.650000  161.020000  156.532800  160.550000   
2021-11-22                1.0  161.680000  165.700000  161.000000  161.020000   

              Adj.Volume  Daily Change  
date                                    
1999-11-01  2.785779e+08     -0.021250  
1999-11-02  3.992356e+08      0.020089  
1999-11-03  3.284627e+08     -0.001071  
1999-11-04  3.790868e+08      0.013929  
1999-11-05  4.168084e+08      0.032946  
...                  ...           ...  
2021-11-16  5.925621e+07      1.060000  
2021-11-17  8.880700e+07      2.495000  
2021-11-18  1.378277e+08      4.160000  
2021-11-19  1.173056e+08      2.900000  
2021-11-22  1.161973e+08     -0.660000  

[5552 rows x 13 columns]

CandeStick

In [53]:
fig = make_subplots()
fig.add_trace(go.Scatter(
    y= apple['Daily Change'])
             )
#fig.add_trace(go.Scatter(
 #   y= apple['Adj.Volume'])
  #           )

fig.show()

In [38]:
area_chart = px.area(y=apple['Daily Change'], x=apple.index)
#area_chart = px.area(y=apple['Volume'], x=apple.index)
area_chart.show()


In [7]:
def candlestick(ticker, name):
    c_candlestick = go.Figure(data = [go.Candlestick(x = ticker.index, 
                                               open = ticker[('Adj_Open')], 
                                               high = ticker[('Adj_High')], 
                                               low = ticker[('Adj_Low')], 
                                               close = ticker[('Adj_Close')])])

    c_candlestick.update_xaxes(
        title_text = 'Date',
        rangeslider_visible = True,
        rangeselector = dict(
            buttons = list([
                dict(count = 1, label = '1M', step = 'month', stepmode = 'backward'),
                dict(count = 6, label = '6M', step = 'month', stepmode = 'backward'),
                dict(count = 1, label = 'YTD', step = 'year', stepmode = 'todate'),
                dict(count = 1, label = '1Y', step = 'year', stepmode = 'backward'),
                dict(step = 'all')])))

    c_candlestick.update_layout(
        title = {
            'text': '%s Share Price (Life Time)' % (name),
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'})

    c_candlestick.update_yaxes(title_text = 'Price', tickprefix = '$')
    return c_candlestick.show()

In [8]:
candlestick(apple, 'Apple')

In [66]:
def candlestick(ticker):
    c_candlestick = go.Figure(data = [go.Candlestick(x = ticker.index, 
                                               open = ticker[('Adj.Open')], 
                                               high = ticker[('Adj.High')], 
                                               low = ticker[('Adj.Low')], 
                                               close = ticker[('Adj.Close')])])


    c_candlestick.update_layout(
        title = {
            'text': '%s Share Price (Life Time)' % (apple.name),
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        xaxis_range=['2020-1-1','2021-11-22'])
    c_candlestick.update_layout(xaxis_rangeslider_visible = False)

    c_candlestick.update_yaxes(title_text = 'Price', tickprefix = '$')
    return c_candlestick.show()

In [67]:
candlestick(apple, 'Apple')

AttributeError: 'DataFrame' object has no attribute 'name'

In [68]:
print(apple.name)

AttributeError: 'DataFrame' object has no attribute 'name'